In [1]:
import os 
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero

dir_ = '../../../data/FM_data/'
group_dir_ = '../../../data/FM_data/groups/high'

In [2]:
file_name = 'normalized_to_rating_filter_track_5_user_50.csv'
df = pd.read_pickle(os.path.join(dir_, file_name[:-3] + 'pkl'))
svd = pd.read_pickle(os.path.join(dir_, 'prediction_svd_top_N_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])

num_user = len(svd['uid'].unique())
num_user

218

In [3]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8', '9', '10', '20']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)

In [4]:
svd['count'] = 1
svd[:5]

,uid,tid,rating,count
0,0,2,2.154104,1
1,0,14,2.175915,1
2,0,15,2.244578,1
3,0,20,2.275416,1
4,0,21,2.088449,1


In [5]:
# Count POP to DF
tid_list = []
pop_list = []
for i in df['tid'].unique():
    tid_list.append(i)
    pop_list.append(len(df[df['tid']==i])/num_user*5)

In [6]:
d = {'tid': tid_list, 'rating': pop_list}
df_pop = pd.DataFrame(data=d)
df_pop = df_pop.sort_values(by=['rating'], ascending=False)
df_pop[:10]

,tid,rating
9147,9147,2.064220
55,55,2.064220
6190,6190,2.018349
555,555,1.972477
1388,1388,1.972477
3825,3825,1.949541
1360,1360,1.949541
48,48,1.949541
355,355,1.949541
5935,5935,1.880734


In [7]:
lambdas = []
for i in range(11):
    lambdas.append(i/10)
lambdas

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

In [15]:
p_list = []
r_list = []
for lambda_ in tqdm(lambdas):
    top_n_size = 4
    top_n_lists_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = df_pop.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
        top_n_lists_g.append(top_n_lists)   
        
    threshold = 0.0
    avg_precisions = []
    avg_recalls = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        precisions = []
        recalls = []
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]
            
            for k in range(len(group)):
                high_rating = 0
                for l in range(top_n_size):
                    uid = group[k]
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    if len(t) > 0 and t.iloc[0]['rating'] > threshold:
                        high_rating += 1

                precision = high_rating / top_n_size
                recall = high_rating / len(test[test['uid'] == uid])
                precisions.append(precision)
                recalls.append(recall)
            

        avg_precision = 0
        for precision in precisions:
            avg_precision += precision
        avg_precision /= len(precisions)
        avg_precisions.append(avg_precision)

        avg_recall = 0
        for recall in recalls:
            avg_recall += recall
        avg_recall /= len(recalls)
        avg_recalls.append(avg_recall)
        
#     print(avg_precisions)
#     print(avg_recalls)
    p_list.append(avg_precisions)
    r_list.append(avg_recalls)

In [16]:
for i in p_list:
    print(i)
print()
for i in r_list:
    print(i)

[0.09939759036144578, 0.10283687943262411, 0.1206896551724138, 0.1159090909090909, 0.12777777777777777, 0.13988095238095238, 0.128125, 0.1111111111111111, 0.15, 0.11875]
[0.17018072289156627, 0.1400709219858156, 0.15517241379310345, 0.13636363636363635, 0.1361111111111111, 0.1636904761904762, 0.1375, 0.1284722222222222, 0.17916666666666667, 0.11875]
[0.22740963855421686, 0.19858156028368795, 0.1896551724137931, 0.18863636363636363, 0.175, 0.19345238095238096, 0.16875, 0.1597222222222222, 0.18333333333333332, 0.11875]
[0.23493975903614459, 0.22163120567375885, 0.2349137931034483, 0.21363636363636362, 0.19722222222222222, 0.22916666666666666, 0.178125, 0.1909722222222222, 0.2, 0.16875]
[0.23493975903614459, 0.2375886524822695, 0.2413793103448276, 0.20681818181818182, 0.18055555555555555, 0.1875, 0.18125, 0.1875, 0.24166666666666667, 0.16875]
[0.23493975903614459, 0.25177304964539005, 0.23706896551724138, 0.17954545454545454, 0.17777777777777778, 0.17261904761904762, 0.1625, 0.16319444444

In [17]:
def dcg_at_k(r, k, method=0):
    r = np.asfarray(r)[:k]
    for i in range(len(r)):
        r[i] = 2**r[i] -1
    
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.


def ndcg_at_k(r, r_max, k, method=0):
    dcg_max = dcg_at_k(r_max, k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

In [18]:
for lambda_ in tqdm(lambdas):
    top_n_lists_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = df_pop.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
        top_n_lists_g.append(top_n_lists)   
        
#     print('lambda = ' + str(lambda_))
    threshold = 0.0
    avg_nDCGs = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        nDCGs = []
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]
            
            for k in range(len(group)):
                truth_rating = []
                uid = group[k]
                for l in range(top_n_size):
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    if len(t) > 0:
                        truth_rating.append(t.iloc[0]['rating'])
                    else:
                        truth_rating.append(0)
                max_rating = test[test['uid']==uid].sort_values(by=['rating'],  ascending=False)['rating'].values[:top_n_size]
                nDCG = ndcg_at_k(truth_rating, max_rating, top_n_size, method=1)
                nDCGs.append(nDCG)  
                
        avg_nDCG = 0
        for nDCG in nDCGs:
            avg_nDCG += nDCG
        avg_nDCG /= len(nDCGs)
        avg_nDCGs.append(avg_nDCG)
        
    print(avg_nDCGs)
#     print('----------------------------------------') 

[0.03508612542342563, 0.039257668444375206, 0.044600099382234734, 0.04310658472562136, 0.04719033549057951, 0.05242242781487224, 0.04980073465179354, 0.04860524809083691, 0.06312408155368802, 0.050230435388119866]
[0.07415887855055311, 0.06205892115276928, 0.06908071548112242, 0.05642811742845822, 0.05707135863444957, 0.059219724136065036, 0.05768982045614255, 0.06049651592639332, 0.07565922358159709, 0.056221188739816984]
[0.10681954728938435, 0.08762665632683182, 0.09136171852193917, 0.07438380832577822, 0.06709858762146849, 0.07114842240137416, 0.07200390267323362, 0.07439893022387542, 0.07893258014709942, 0.04824032250670131]
[0.10995424243324842, 0.10791983323221883, 0.11136446452687243, 0.09409657244530137, 0.07960991709601367, 0.09519757127270288, 0.07747318623850484, 0.07807868329234752, 0.0873554231657345, 0.05399386872232732]
[0.11059748921167889, 0.11384438603735045, 0.11856903724465025, 0.09159806034124075, 0.06913535474262134, 0.0903187914617911, 0.0771597193052332, 0.0825